In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from ff import FF, FFLayer
from data import MNIST

In [ ]:
batch_size_train = 32
batch_size_test = 32

In [ ]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./datasets/MNIST/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

In [ ]:
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./datasets/MNIST/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [ ]:
threshold = 1.5
epochs = 50
model = FF()
model.add_layer(FFLayer(nn.Linear(784, 500), optimizer=torch.optim.Adam, epochs=epochs, threshold=threshold, activation=nn.ReLU(), lr=0.01, positive_lr=0.005, negative_lr=0.005))
model.add_layer(FFLayer(nn.Linear(500, 500), optimizer=torch.optim.Adam, epochs=epochs, threshold=threshold, activation=nn.ReLU(), lr=0.01, positive_lr=0.005, negative_lr=0.005))
model.add_layer(FFLayer(nn.Linear(500, 500), optimizer=torch.optim.Adam, epochs=epochs, threshold=threshold, activation=nn.ReLU(), lr=0.01, positive_lr=0.005, negative_lr=0.005))

In [ ]:
model.train()
for i, (x, y) in enumerate(train_loader):
    x_pos, _ = MNIST.overlay_y_on_x(x, y)
    rnd = torch.randperm(x.size(0))
    x_neg, _ = MNIST.overlay_y_on_x(x, y[rnd])
    #print(x_pos.requires_grad, x_neg.requires_grad)

    # Nu ii place deloc daca fac una dupa alta
    #losses = model.forward_positive(x_pos)
    #losses += model.forward_negative(x_neg)
    #print(losses/2)

    losses = model(x_pos, x_neg)
    print(losses)

In [ ]:
predictions, real = MNIST.predict(test_loader, model)
print("Accuracy ", np.sum(predictions == real)/len(real))